# 1. Data Preparation
Resave data as a set of tiff files in order to match Cell Tracking Challenge conventions which are expected by EmbedTrack.

In [3]:
import os

import numpy as np
from tifffile import imwrite

from deepcell_tracking.isbi_utils import trk_to_isbi
from deepcell_tracking.trk_io import load_trks

In [9]:
source_data = '../../data/test.trks'
data_dir = 'data'
raw_dir = os.path.join(data_dir, 'raw')
seg_dir = os.path.join(raw_dir, 'Analysis/Segmentation_GT')
gt_dir = os.path.join(data_dir, 'GT')

for d in [raw_dir, seg_dir, gt_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

Load the test split of the tracking data

In [5]:
data = load_trks(source_data)

Convert each batch of the test split to the standard ISBI format which is compatible with most of the models that we will tst.

In [10]:
for batch_no in range(len(data['lineages'])):
    # Build subdirectories for data
    raw_subdir = os.path.join(raw_dir, '{:03}'.format(batch_no + 1))
    gt_subdir = os.path.join(gt_dir, '{:03}'.format(batch_no + 1))
    seg_subdir = os.path.join(seg_dir, '{:03}'.format(batch_no + 1))
    
    # Create directories if needed
    for d in (raw_subdir, gt_subdir, seg_subdir):
        if not os.path.exists(d):
            os.makedirs(d)
                
    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineages = data['lineages'][batch_no]
    
    # Need to translate lineages and adjust images to match restrictive ISBI format
    # Prepare output txt
    text_file = os.path.join(gt_subdir, 'man_track.txt')
    df = trk_to_isbi(lineages)
    df.to_csv(text_file, sep=' ', header=False)
    
    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    # We assume here that the empty frames are at the end of the movie (padding rather than skipped)
    movie_len = len(good_frames)
    
    # Save each frame of the movie as an individual tif
    channel = 0 # These images should only have one channel
    for i in range(movie_len):
        name_raw = os.path.join(raw_subdir, 't{:03}.tif'.format(i))
        name_seg = os.path.join(seg_subdir, 't{:03}.tif'.format(i))
        name_gt = os.path.join(gt_subdir, 't{:03}.tif'.format(i))
        
        imwrite(name_raw, X[i, ..., channel])
        imwrite(name_seg, y[i, ..., channel].astype('uint16'))
        imwrite(name_gt, y[i, ..., channel].astype('uint16'))

# 2. Tracking

The steps for tracking will be performed in Matlab and the Baxter Algorithm GUI. You will need access to Matlab 2019b. The instructions below outline the general workflow for generating benchmarking results using the Baxter algorithm. However additional support for using the software is available in the user guide located in the KTH-SE folder (`KTH-SE/UserGuide/UserGuide.pdf`).

1. Open the KTH-SE folder in Matlab
2. Open `BaxterAlgorithms.m` in the Matlab editor and then hit Run. This should open a GUI.
3. Within the GUI, select `Files > Open experiment > Browse...`. Navigate to the `data/raw` folder located within this Baxter folder.
4. Ensure that all files are selected in the list on the right hand side of the GUI.
5. Select `Settings > Load Settings (Browse for files)`. Navigate to `SegmentationSettings.csv` within the Baxter folder. In the following prompts, ensure that segmentations are applied to all images.
6. Select `Automated > Track`. In the pop-up window, press `Start`. 
7. Select `File > Export tracks to CTC format > RES tracks` and follow the prompts in order to export the tracking results to the standard CTC format.

# 3. Evaluation

Please note the name of the folder that Baxter exported into. It should take the form `data/raw/Analysis/CellData_*`. You will need to substitute the name generated in your export in the cell below.

In [1]:
import glob
import os
import re

import numpy as np
import pandas as pd
from tifffile import imread

from deepcell_tracking.metrics import TrackingMetrics

In [2]:
export_dir = 'data/raw/Analysis/CellData_221103_175542'

In [3]:
data_dir = 'data/GT'
results_dir = os.path.join(export_dir, 'RES')

data_ids = os.listdir(data_dir)

node_match_threshold = 0.6

In [5]:
benchmarks = []

for data_id in data_ids:
    gt_dir = os.path.join(data_dir, f'{data_id}')
    res_dir = os.path.join(results_dir, f'{data_id}_RES')

    metrics = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir)
    benchmarks.append({
        'data_id': data_id,
        **metrics.stats
    })
        
df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

missed node 15_17 division completely
missed node 19_20 division completely
22_5 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 29_17 division completely
missed node 59_10 division completely
missed node 60_34 division completely
76_7 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 79_9 division completely
missed node 80_9 division completely
missed node 82_17 division completely
missed node 83_18 division completely
corrected division 79_9 as a frameshift division not an error
corrected division 82_17 as a frameshift division not an error
missed node 1_48 division completely
missed node 5_6 division completely
missed node 10_66 division completely
missed node 15_66 division completely
missed node 25_57 division completely
30_37 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 55_27 division completely
missed node 76_44 division completely
missed node 94_36 division completely
missed node 104_34 division complet